### データベースからEVのデータを取得する

In [97]:
import pandas as pd
import pyodbc

In [98]:
def get_leafspy_data():
    driver='{SQL Server}'
    server = # server name
    database = # database name
    trusted_connection='yes'
    connect= pyodbc.connect('DRIVER='+driver+';SERVER='+server+';DATABASE='+database+';PORT=1433;Trusted_Connection='+trusted_connection+';')
    cursor = connect.cursor()
    sql_query = """ WITH BASE AS(
                    SELECT 
                    ECOLOG.CAR_ID,
                    ECOLOG.DRIVER_ID,
                    ECOLOG.SENSOR_ID,
                    ECOLOG.JST,
                    ECOLOG.SPEED,
                    ECOLOG.[LATITUDE],
                    ECOLOG.[LONGITUDE],
                    ECOLOG.[TRIP_DIRECTION],
                    ECOLOG.TRIP_ID,
                    LEAFSPY.[PACK_VOLTS],
                    LEAFSPY.[PACK_AMP],
                    LEAFSPY.[PACK_AMP]*[PACK_VOLTS] AS EV_POWER,
                    LEAFSPY.[GIDS],
                    LEAFSPY.[SOC],
                    LEAFSPY.[PACK_AMP]*[PACK_VOLTS] - AC_POWER_250W*250 - AUX_POWER_100W*100 AS RUN_POWER,
                    LEAFSPY.AUX_POWER_100W,
                    LEAFSPY.AC_POWER_250W,
                    LEAFSPY.ESTIMATED_AC_POWER_50W,
                    LEAFSPY.ESTIMATED_HEATER_POWER_250W,
                    (LEAFSPY.[PACK_T1_C] +LEAFSPY.[PACK_T2_C]+LEAFSPY.[PACK_T4_C])/3　AS BETTERY_TEMPERATURA,
                    LEAFSPY.[AMBIENT],
                    LEAFSPY.[SOH]
                    ,rank()over(partition by ECOLOG.TRIP_ID order by cast(ECOLOG.JST as time)) AS RUN_ID
                    FROM # table name AS ECOLOG
                    INNER JOIN  # table name AS LEAFSPY
                    ON LEAFSPY.JST = ECOLOG.JST
                    ),

                    result1 AS (SELECT *,
                    ABS(DATEDIFF(SECOND, lEAD(JST, 1) OVER (ORDER BY JST), JST)) AS SECONDS_DIFF
                    FROM BASE
                    WHERE (JST BETWEEN '2020-09-01' AND '2022-12-15')
                    AND (TRIP_DIRECTION = 'homeward' OR TRIP_DIRECTION = 'outward')
                    AND (SENSOR_ID = '31' OR SENSOR_ID = '34'OR SENSOR_ID = '39')
                    ),
                    
                    result2 AS (SELECT *,
                    ABS(DATEDIFF(SECOND, lEAD(JST, 1) OVER (ORDER BY JST), JST)) AS SECONDS_DIFF
                    FROM BASE
                    WHERE (JST BETWEEN '2023-02-01' AND '2023-09-01')
                    AND (TRIP_DIRECTION = 'homeward' OR TRIP_DIRECTION = 'outward')
                    AND (SENSOR_ID = '31' OR SENSOR_ID = '34'OR SENSOR_ID = '39')
                    )
                    
                    SELECT *
                    FROM result1
                    UNION ALL
                    SELECT *
                    FROM result2
                    ORDER BY JST """
        
    cursor.execute(sql_query) 
    rows = cursor.fetchall()    
    rows_dicts = [dict(zip([column[0] for column in cursor.description], row)) for row in rows]
    df = pd.DataFrame(rows_dicts)
    cursor.close()
    connect.close()
    
    return df

In [99]:
def get_weather_data(place_id):
    driver='{SQL Server}'
    server = # server name
    database = # database name
    trusted_connection='yes'
    connect= pyodbc.connect('DRIVER='+driver+';SERVER='+server+';DATABASE='+database+';PORT=1433;Trusted_Connection='+trusted_connection+';')
    cursor = connect.cursor()
    sql_query = """  SELECT [PLACE],[DATETIME],[LAST_10MIN_DATETIME],[TEMPERATURE],[HUMIDITY],[WIND_SPEED],[WIND_DIRECTION]
                     FROM # table name
                     WHERE [DATETIME] BETWEEN '2020-09-01' AND '2023-09-01'
                     AND [PLACE] = {}
                     ORDER BY [DATETIME] """.format(place_id)
        
    cursor.execute(sql_query) 
    rows = cursor.fetchall()    
    rows_dicts = [dict(zip([column[0] for column in cursor.description], row)) for row in rows]
    df = pd.DataFrame(rows_dicts)
    cursor.close()
    connect.close()
    
    return df

In [100]:
def get_illuminacne_data():
    driver='{SQL Server}'
    server = # server name
    database = # database name
    trusted_connection='yes'
    connect= pyodbc.connect('DRIVER='+driver+';SERVER='+server+';DATABASE='+database+';PORT=1433;Trusted_Connection='+trusted_connection+';')
    cursor = connect.cursor()
    sql_query = """ SELECT *
                    FROM # table name
                    WHERE [DATETIME] BETWEEN '2020-09-01' AND '2023-09-01'
                    ORDER BY [DATETIME]"""
        
    cursor.execute(sql_query)
    rows = cursor.fetchall()
    rows_dicts = [dict(zip([column[0] for column in cursor.description], row)) for row in rows]
    df = pd.DataFrame(rows_dicts)
    cursor.close()
    connect.close()
    
    return df

In [101]:
weather_yokohama = get_weather_data(47670)
weather_ebina = get_weather_data(460388)
leafspy = get_leafspy_data()
illuminance = get_illuminacne_data()

weather_yokohama.to_csv('weather_yokohama.csv')
weather_ebina.to_csv('weather_ebina.csv')
leafspy.to_csv('leafspy.csv')
illuminance.to_csv('illuminance.csv')